In [2]:
import torch

![image](https://www.ibm.com/content/dam/connectedassets-adobe-cms/worldwide-content/creative-assets/s-migr/ul/g/3f/b4/mha-mqa-and-gqa.component.xl-retina.ts=1744898975663.png/content/adobe-cms/us/en/think/topics/grouped-query-attention/jcr:content/root/table_of_contents/body-article-8/image_700444495)

In [ ]:
B = 25 # batch size 
T = 50 # sequence length for Q 
S = T # sequence length for K,V 
D = 8*50 # d_model embedding dim 
N = 8 # no of query heads 
K = 4 # no of key, value heads 
H = D // N # attention head dim 
G = N // K # q heads per kv head 

x = torch.rand((B, T, D))

Q_w = torch.rand((D, N, H))
K_w = torch.rand((D, K, H))
V_w = torch.rand((D, K, H))
output_w = torch.rand((N, H, D))

query = torch.einsum("BTD,DNH->BTNH", x, Q_w)   # (B,T,N,H)
key   = torch.einsum("BTD,DKH->BTKH", x, K_w)   # (B,T,K,H)
value = torch.einsum("BTD,DKH->BTKH", x, V_w)   # (B,T,K,H)

query = query.reshape(B, T, K, G, H)            # (B,T,K,G,H)

attn_logits = torch.einsum("BTKGH,BSKH->BTSKG", query, key)  # (B,T,S,K,G)
scale = torch.sqrt(torch.tensor(H, dtype=attn_logits.dtype, device=attn_logits.device))
attn_logits = attn_logits / scale

causal = torch.ones(T, S, dtype=torch.bool, device=attn_logits.device).tril().view(1, T, S, 1, 1)
attn_logits = attn_logits.masked_fill(~causal, float("-inf"))

attn_probs = torch.softmax(attn_logits, dim=2)               # (B,T,S,K,G)

context = torch.einsum("BTSKG,BSKH->BTKGH", attn_probs, value)  # (B,T,K,G,H)
context = context.reshape(B, T, N, H)                           # (B,T,N,H)

outputs = torch.einsum("BTNH,NHD->BTD", context, output_w)      # (B,T,D)